## UTILITY

In [1]:
import random
import numpy as np
import string
import re
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, Model, Input

# DATA DOWNLOAD AND REVIEW

**Download data from the source**

In [2]:
!wget https://www.manythings.org/anki/ita-eng.zip
!unzip -q ita-eng.zip

--2024-08-02 00:53:10--  https://www.manythings.org/anki/ita-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8326901 (7.9M) [application/zip]
Saving to: ‘ita-eng.zip’

ita-eng.zip         100%[===================>]   7.94M  4.41MB/s    in 1.8s    

2024-08-02 00:53:13 (4.41 MB/s) - ‘ita-eng.zip’ saved [8326901/8326901]



**Open and read the file and create pairs**

In [6]:
text_file = '/content/ita.txt'
with open(text_file) as f:
    lines = f.read().split('\n')[:-1]

text_pairs = []

# Create English and Italian pairs of sentences.
for line in lines:
    english, italian, ignore = line.split('\t')
    italian = '[start] ' + italian + ' [end]'
    text_pairs.append((english, italian))

In [10]:
# Randonly display a pair.
print(random.choice(text_pairs))

# Display the count of pairs.
print(len(text_pairs))

('Tom asked Mary to be his secretary.', '[start] Tom chiese a Mary di essere la sua segretaria. [end]')
377937


# DATA SPLIT

In [11]:
# Randomly shuffle the pairs.
random.shuffle(text_pairs)

num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples

# Split the pairs for training, validation and testing.
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

# DATA PROCESSING

**Vectorizing the English and Italian text pairs**

In [13]:
# Strip all the punctuations.
strip_chars = string.punctuation

#  Preserve the "[start]" and "[end]" tokens that we’ve inserted.
strip_chars = strip_chars.replace('[', '')
strip_chars = strip_chars.replace(']', '')

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f'[{re.escape(strip_chars)}]', '')

# Top n_words to consider.
vocab_size = 15000

# Restrict sentences to 20 words.
sequence_length = 20

# Create a TextVectorization layer for the source text
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,  # Maximum number of unique words to consider
    output_mode='int',  # Convert words to integer indices
    output_sequence_length=sequence_length,  # Length of the output sequences
)

# Create a TextVectorization layer for the target text
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,  # Maximum number of unique words to consider
    output_mode='int',  # Convert words to integer indices
    output_sequence_length=sequence_length + 1,  # Length of the output sequences (1 more than source)
    standardize=custom_standardization,  # Apply a custom standardization process
)

train_english_texts = [pair[0] for pair in train_pairs]
train_italian_texts = [pair[1] for pair in train_pairs]

# Learn the vocabulary of each language.
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_italian_texts)

**Preparing datasets for the translation task**

In [15]:
# Set the batch size for the dataset
batch_size = 64

def format_dataset(eng, ita):
    # Apply text vectorization to the English and Italian texts
    eng = source_vectorization(eng)
    ita = target_vectorization(ita)

    # Return a dictionary with English and Italian inputs and Italian outputs
    return ({
        'english': eng,             # Input: English text
        'italian': ita[:, :-1],     # Input: Italian text (excluding the last token)
    }, ita[:, 1:])                  # Output: Italian text (excluding the first token)

def make_dataset(pairs):
    # Unpack the pairs of English and Italian texts
    eng_texts, ita_texts = zip(*pairs)

    # Convert texts to lists
    eng_texts = list(eng_texts)
    ita_texts = list(ita_texts)

    # Create a TensorFlow dataset from the texts
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, ita_texts))

    # Group texts into batches
    dataset = dataset.batch(batch_size)

    # Apply the format_dataset function to each batch
    dataset = dataset.map(format_dataset, num_parallel_calls=4)

    # Shuffle, prefetch, and cache the dataset for better performance
    return dataset.shuffle(2048).prefetch(16).cache()

# Create training and validation datasets
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [17]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['italian'].shape: {inputs['italian'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


# SEQUENCE-TO-SEQUENCE LEARNING WITH RNN

**GRU-based encoder**

In [18]:
embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

**GRU-based decoder and the end-to-end model**

In [19]:
past_target = keras.Input(shape=(None,), dtype="int64", name="italian")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)

seq2seq_rnn = keras.Model([source, past_target], target_next_step)

**Training our recurrent sequence-to-sequence model**

In [20]:
seq2seq_rnn.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
4134/4134 ━━━━━━━━━━━━━━━━━━━━ 421s 100ms/step - accuracy: 0.1898 - loss: 3.3509 - val_accuracy: 0.2744 - val_loss: 1.1147
Epoch 2/15
4134/4134 ━━━━━━━━━━━━━━━━━━━━ 416s 101ms/step - accuracy: 0.2711 - loss: 1.1606 - val_accuracy: 0.2910 - val_loss: 0.8118
Epoch 3/15
4134/4134 ━━━━━━━━━━━━━━━━━━━━ 416s 101ms/step - accuracy: 0.2893 - loss: 0.8205 - val_accuracy: 0.2961 - val_loss: 0.7375
Epoch 4/15
4134/4134 ━━━━━━━━━━━━━━━━━━━━ 416s 101ms/step - accuracy: 0.2979 - loss: 0.6861 - val_accuracy: 0.2983 - val_loss: 0.7079
Epoch 5/15
4134/4134 ━━━━━━━━━━━━━━━━━━━━ 417s 101ms/step - accuracy: 0.3026 - loss: 0.6146 - val_accuracy: 0.3005 - val_loss: 0.6968
Epoch 6/15
4134/4134 ━━━━━━━━━━━━━━━━━━━━ 416s 101ms/step - accuracy: 0.3072 - loss: 0.5682 - val_accuracy: 0.3015 - val_loss: 0.6928
Epoch 7/15
4134/4134 ━━━━━━━━━━━━━━━━━━━━ 416s 101ms/step - accuracy: 0.3097 - loss: 0.5378 - val_accuracy: 0.3008 - val_loss: 0.6941
Epoch 8/15
4134/4134 ━━━━━━━━━━━━━━━━━━━━ 416s 101ms/step - ac

**Translating new sentences with our RNN encoder and decoder**

In [21]:
ita_vocab = target_vectorization.get_vocabulary()
ita_index_lookup = dict(zip(range(len(ita_vocab)), ita_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = ita_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Tom tried to explain everything to Mary.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[start] tom ha cercato di spiegare tutto a mary [end]
-
Tom is afraid of you.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[start] tom ha paura di voi [end]
-
It took me some time to persuade her.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━

**Translating a random sentence**

In [22]:
input_sentence = 'The day is beautiful.'
print(decode_sequence(input_sentence))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[start] la giornata è bella [end]


# LIMITATIONS

- The encoder state vector(s) must hold the entire source sequence, limiting the size and complexity of translatable sentences. It's like translating a sentence from memory without rereading it.

- RNNs struggle with long sequences because they gradually forget earlier information. By the 100th token, there's little left from the start of the sequence.